<a href="https://colab.research.google.com/github/Fabiorsi1979/DS-Comunidade/blob/main/SQL_verif_tp_coluna_ope_matematicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Verificação de tipo da Coluna e Operações Matemáticas em condicionais.

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
path = '/content/drive/MyDrive/db_olist.sqlite'

In [3]:
import sqlite3
import pandas as pd

In [7]:
def Mat_cond_sql(*args):
  conexao = sqlite3.connect(path)
  cursor = conexao.cursor()

  cursor.execute(consulta)
  dados = cursor.fetchall()

  cursor.close()
  conexao.close()

  consulta_sql = pd.DataFrame(data=dados)

  return consulta_sql

#### 1. Qual o valor da média ponderada das avaliações dos produtos que foram comprados a partir do dia 1 de Janeiro de 2018.
##### - Nota 5 → Peso 0.2
##### - Nota 4 → Peso 0.1
##### - Nota 3 → Peso 0.3
##### - Nota 2 → Peso 0.3
##### - Nota 1 → Peso 0.1
##### - Nota 0 → Peso 0.0

In [30]:
consulta = """
SELECT
    table1.SOMA_pesos_multiplicados / table1.Soma_pesos AS Media_ponderada
  FROM (SELECT
          SUM (CASE
                  WHEN ow.review_score IN (1,4) THEN (cast(ow.review_score AS REAL) * 0.1)
                  WHEN ow.review_score IN (2,3) THEN (cast(ow.review_score AS REAL) * 0.3)
                  WHEN ow.review_score = 5 THEN (cast(ow.review_score AS REAL) * 0.2)
                  ELSE (cast(ow.review_score AS REAL) * 0.0)
          END ) AS SOMA_pesos_multiplicados,
          SUM (CASE
                  WHEN ow.review_score IN (1,4) THEN 0.1
                  WHEN ow.review_score IN (2,3) THEN 0.3
                  WHEN ow.review_score = 5 THEN 0.2
                  ELSE 0.0
          END ) AS SOMA_pesos
        FROM orders o LEFT JOIN order_reviews ow ON ( ow.order_id = o.order_id )
        WHERE DATE(o.order_approved_at)>= '2018-01-01' AND DATE(o.order_approved_at) IS NOT NULL) AS table1
"""
resultado = Mat_cond_sql (consulta)
resultado.columns = ["Media Ponderada"]
resultado

,Media Ponderada
0,4.204427


#### 2. Calcule a média ponderada por mês, do exercício anterior

In [36]:
consulta = """
SELECT
    table2.Mes,
    table2.SOMA_pesos_multiplicados / table2.Soma_pesos AS Media_ponderada
  FROM (SELECT
              STRFTIME ('%m',o.order_approved_at) As Mes,
              SUM (CASE
                      WHEN ow.review_score IN (1,4) THEN (cast(ow.review_score AS REAL) * 0.1)
                      WHEN ow.review_score IN (2,3) THEN (cast(ow.review_score AS REAL) * 0.3)
                      WHEN ow.review_score = 5 THEN (cast(ow.review_score AS REAL) * 0.2)
                      ELSE (cast(ow.review_score AS REAL) * 0.0)
              END) AS SOMA_pesos_multiplicados,
              SUM (CASE
                      WHEN ow.review_score IN (1,4) THEN 0.1
                      WHEN ow.review_score IN (2,3) THEN 0.3
                      WHEN ow.review_score = 5 THEN 0.2
                      ELSE 0.0
              END) AS SOMA_pesos
        FROM orders o LEFT JOIN order_reviews ow ON ( ow.order_id = o.order_id )
        WHERE DATE(o.order_approved_at)>= '2015-01-01' AND DATE(o.order_approved_at) IS NOT NULL
        GROUP BY Mes ) AS Table2
"""
resultado = Mat_cond_sql (consulta)
resultado.columns = ["Mes","Media Ponderada"]
resultado

,Mes,Media Ponderada
0,01,4.148357
1,02,4.023605
2,03,3.995306
3,04,4.217711
4,05,4.264809
5,06,4.312822
6,07,4.312941
7,08,4.338235
8,09,4.273307
9,10,4.202776


#### 3.  Selecione os seguintes categorias de produtos: livros técnicos, petshop, pc gamer, tablets impressão imagem, fashion esports, perfumaria, telefonia, beleza saude, ferramentas jardim.
##### Calcule o desconto total, o desconto médio, o desconto mínimo, o desconto máximo e quantidade de produtos com desconto, dado pela loja ao reduzir o valor dos produtos da seguinte categoria:
##### - Livros técnicos - 10% de desconto
##### - Pet shop - 20% de desconto
##### - PC gamer - 50% de desconto
##### - Tablets - 90% de desconto
##### - Fashion Esports - 40% de desconto


In [44]:
consulta ="""
SELECT
      SUM(table3.New_price)   AS Total_price,
      AVG(table3.New_price)   As Media_price,
      MIN(table3.New_price)   AS Min_price,
      MAX(table3.New_price)   AS Max_price,
      COUNT(table3.Produtos)  AS Qtde_prod
FROM (SELECT
          oi.product_id AS Produtos,
          p.product_category_name,
          oi.price,
          CASE
              WHEN p.product_category_name ='livros_tecnicos'          THEN CAST((oi.price * 0.10) AS REAL)
              WHEN p.product_category_name ='pet_shop'                 THEN CAST((oi.price * 0.20) AS REAL)
              WHEN p.product_category_name ='pc_gamer'                 THEN CAST((oi.price * 0.50) AS REAL)
              WHEN p.product_category_name ='tablets_impressao_imagem' THEN CAST((oi.price * 0.90) AS REAL)
              WHEN p.product_category_name ='fashion_esporte'          THEN CAST((oi.price * 0.40) AS REAL)
              ELSE 0
          END AS New_price
      FROM products p LEFT JOIN order_items oi ON ( p.product_id = oi.product_id )
      WHERE p.product_category_name IN ("livros_tecnicos","pet_shop","pc_gamer","tablets_impressao_imagem",
                              "fashion_esporte","perfumaria","telefonia","beleza_saude","ferramentas_jardim")) AS table3
"""
resultado = Mat_cond_sql(consulta)
resultado.columns=["Desconto Total", "Desconto Médio"," Menor Desconto","Maior Desconto","Qtde Produtos únicos c/desc"]
resultado

,Desconto Total,Desconto Médio,Menor Desconto,Maior Desconto,Qtde Produtos únicos c/desc
0,53169.036,2.186497,0,800.991,24317


##### 4. Usando o exercícios anterior, crie uma consulta que calcule o desconto total, o desconto médio, o desconto mínimo, o desconto máximo e quantidade de produtos com desconto por categoria


In [46]:
consulta ="""
SELECT
      table3.Category,
      SUM(table3.New_price)   AS Total_price,
      AVG(table3.New_price)   As Media_price,
      MIN(table3.New_price)   AS Min_price,
      MAX(table3.New_price)   AS Max_price,
      COUNT(table3.Produtos)  AS Qtde_prod
FROM (SELECT
          oi.product_id AS Produtos,
          p.product_category_name AS Category,
          oi.price,
          CASE
              WHEN p.product_category_name ='livros_tecnicos'          THEN CAST((oi.price * 0.10) AS REAL)
              WHEN p.product_category_name ='pet_shop'                 THEN CAST((oi.price * 0.20) AS REAL)
              WHEN p.product_category_name ='pc_gamer'                 THEN CAST((oi.price * 0.50) AS REAL)
              WHEN p.product_category_name ='tablets_impressao_imagem' THEN CAST((oi.price * 0.90) AS REAL)
              WHEN p.product_category_name ='fashion_esporte'          THEN CAST((oi.price * 0.40) AS REAL)
              ELSE 0
          END AS New_price
      FROM products p LEFT JOIN order_items oi ON ( p.product_id = oi.product_id )
      WHERE p.product_category_name IN ("livros_tecnicos","pet_shop","pc_gamer","tablets_impressao_imagem",
                              "fashion_esporte","perfumaria","telefonia","beleza_saude","ferramentas_jardim")) AS table3
GROUP BY table3.Category
"""
resultado = Mat_cond_sql(consulta)
resultado.columns=["Categoria","Desconto Total", "Desconto Médio"," Menor Desconto","Maior Desconto","Qtde Produtos únicos c/desc"]
resultado

,Categoria,Desconto Total,Desconto Médio,Menor Desconto,Maior Desconto,Qtde Produtos únicos c/desc
0,beleza_saude,0.000,0.000000,0.000,0.000,9670
1,fashion_esporte,847.804,28.260133,9.996,90.000,30
2,ferramentas_jardim,0.000,0.000000,0.000,0.000,4347
3,livros_tecnicos,1909.606,7.152082,0.995,38.493,267
4,pc_gamer,772.975,85.886111,64.995,119.500,9
5,perfumaria,0.000,0.000000,0.000,0.000,3419
6,pet_shop,42863.082,22.014937,0.580,499.600,1947
7,tablets_impressao_imagem,6775.569,81.633361,13.410,800.991,83
8,telefonia,0.000,0.000000,0.000,0.000,4545
